In [0]:
df = spark.read.option("multiLine", True).json("/FileStore/tables/Nested.json")
df.printSchema()
df.show(truncate=False)

df = df.drop('attribute1')


fields_to_remove = {"endGradeSeparation", "formsPartOfPath"}
fields_to_keep = [f.name for f in df.schema["pathLinkInfo"].dataType.fields if f.name not in fields_to_remove]

df = df.withColumn(
    "pathLinkInfo",
    struct(*[col(f"pathLinkInfo.{field}") for field in fields_to_keep])
)

df.printSchema()

elevationGain_fields = df.schema["pathLinkInfo"].dataType["elevationGain"].dataType.fields
elevationGain_fields_to_keep = [f.name for f in elevationGain_fields if f.name != "elevationAgainstDirection"]

df = df.withColumn(
    "pathLinkInfo",
    struct(
        *[
            col(f"pathLinkInfo.{field}") if field != "elevationGain" else
            struct(*[col(f"pathLinkInfo.elevationGain.{ef}") for ef in elevationGain_fields_to_keep]).alias("elevationGain")
            for field in fields_to_keep
        ]
    )
)


df.printSchema()

root
 |-- CaptureDate: string (nullable = true)
 |-- CaptureID: long (nullable = true)
 |-- FID: string (nullable = true)
 |-- Geometry: string (nullable = true)
 |-- attribute1: string (nullable = true)
 |-- changeType_source: string (nullable = true)
 |-- identifier: long (nullable = true)
 |-- pathLinkInfo: struct (nullable = true)
 |    |-- alternateName: string (nullable = true)
 |    |-- captureSpecification: string (nullable = true)
 |    |-- cycleFacility: string (nullable = true)
 |    |-- elevationGain: struct (nullable = true)
 |    |    |-- elevationAgainstDirection: double (nullable = true)
 |    |    |-- elevationInDirection: long (nullable = true)
 |    |-- endGradeSeparation: long (nullable = true)
 |    |-- endNode: string (nullable = true)
 |    |-- fictitious: boolean (nullable = true)
 |    |-- formOfWay: string (nullable = true)
 |    |-- formsPartOfPath: string (nullable = true)
 |    |-- formsPartOfStreet: array (nullable = true)
 |    |    |-- element: string (c